In [1]:
import datetime
import os
import pickle
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
from sklearn.inspection import PartialDependenceDisplay, partial_dependence
from sklearn.metrics import (ConfusionMatrixDisplay, accuracy_score, precision_recall_curve, roc_auc_score, roc_curve)
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import label_binarize
from xgboost import XGBClassifier

import shap

%config InlineBackend.figure_format = 'retina'

In [2]:
from notebook import notebookapp
import urllib
import json
import ipykernel
from shutil import copy2

def notebook_path():
    """Returns the absolute path of the Notebook or None if it cannot be determined
    NOTE: works only when the security is token-based or there is also no password
    """
    connection_file = os.path.basename(ipykernel.get_connection_file())
    kernel_id = connection_file.split('-', 1)[1].split('.')[0]

    for srv in notebookapp.list_running_servers():
        try:
            if srv['token']=='' and not srv['password']:  # No token and no password, ahem...
                req = urllib.request.urlopen(srv['url']+'api/sessions')
            else:
                req = urllib.request.urlopen(srv['url']+'api/sessions?token='+srv['token'])
            sessions = json.load(req)
            for sess in sessions:
                if sess['kernel']['id'] == kernel_id:
                    return os.path.join(srv['notebook_dir'],sess['notebook']['path'])
        except:
            pass  # There may be stale entries in the runtime directory 
    return None


def copy_current_nb(new_name):
    nb = notebook_path()
    if nb:
        new_path = os.path.join(os.path.dirname(nb), new_name+'.ipynb')
        copy2(nb, new_path)
    else:
        print("Current notebook path cannot be determined.")

In [3]:
df = pd.read_csv('Data/cover_type_engineered.csv')

In [4]:
df = df.loc[:, [col for col in df if not col.startswith('Cover_Type_')]]
# df = df.loc[(df['Cover_Type'] == 1) | (df['Cover_Type'] == 2)]

X = df.drop(columns=['Cover_Type', 'Aspect_Sector'])
y_train = df['Cover_Type'] - 1

In [5]:
X = X.loc[:, [col for col in X if not col.startswith('Soil_Type')]]

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X)

In [ ]:
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M")

warnings.filterwarnings("ignore", category=FutureWarning, module="pandas.api.types")

# Assuming X and y are defined
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.33, random_state=42)

# Define the estimator
estimator = XGBClassifier(tree_method="hist")

# Define hyperparameters for tuning
hyperparameters = {
    "n_estimators": stats.randint(50, 150),  # Increased range
    "learning_rate": stats.uniform(0.05, 0.1),  # Adjusted range
    "colsample_bytree": stats.uniform(0.6, 0.4),
    "colsample_bylevel": stats.uniform(0.6, 0.4),
    "colsample_bynode": stats.uniform(0.6, 0.4),
    "max_depth": stats.randint(15, 50),  # Increased range
    "subsample": stats.uniform(0.6, 0.4),
    "gamma": stats.uniform(0, 2),  # Adjusted range
    "reg_lambda": stats.uniform(0, 2),  # Adjusted range
    "reg_alpha": stats.uniform(0, 2),  # Adjusted range
}

# Define the RandomizedSearchCV parameters
random_search = RandomizedSearchCV(estimator, 
                                   param_distributions=hyperparameters, 
                                   scoring='accuracy',
                                   return_train_score=True,
                                   n_iter=500, 
                                   cv=5, 
                                   verbose=10, 
                                   n_jobs=-1)

# Fit the RandomizedSearchCV
try:
    random_search.fit(X_train, y_train)  # Assuming X_train and y_train are defined
    print("Best parameters found:", random_search.best_params_)
    print("Best score found:", random_search.best_score_)

    
    # Save results
    results_path = f"./tuning_results/tuning_xgb/{timestamp}"
    if not os.path.exists(results_path):
        os.makedirs(results_path)
        
    # Saving cross-validation results
    cv_results = pd.DataFrame(random_search.cv_results_)
    cv_results_file = f"{timestamp}_results.csv"
    cv_results.to_csv(os.path.join(results_path, cv_results_file), index=False)
    
    # Save .ipynb
    copy_current_nb(os.path.join(results_path, 'Evaluation_Notebook'))
    
    # Save Model
    file_name = f"xgb_{timestamp}.pkl"
    pickle.dump(random_search, open(os.path.join(results_path, file_name), "wb"))
        

except Exception as e:
    print(f"An error occurred during model optimization: {e}")


Fitting 5 folds for each of 500 candidates, totalling 2500 fits
[CV 2/5; 1/500] START colsample_bylevel=0.9711041112967383, colsample_bynode=0.8260058929866001, colsample_bytree=0.8910774836315021, gamma=0.9097375308994864, learning_rate=0.09241782445712447, max_depth=23, n_estimators=56, reg_alpha=1.468026767557701, reg_lambda=1.8702812504091426, subsample=0.8123740778035529
[CV 2/5; 1/500] END colsample_bylevel=0.9711041112967383, colsample_bynode=0.8260058929866001, colsample_bytree=0.8910774836315021, gamma=0.9097375308994864, learning_rate=0.09241782445712447, max_depth=23, n_estimators=56, reg_alpha=1.468026767557701, reg_lambda=1.8702812504091426, subsample=0.8123740778035529;, score=(train=0.948, test=0.814) total time=   0.5s
[CV 1/5; 4/500] START colsample_bylevel=0.8190588915005117, colsample_bynode=0.7166221384282041, colsample_bytree=0.7856184585507209, gamma=0.866275196184985, learning_rate=0.10134444781341637, max_depth=35, n_estimators=103, reg_alpha=1.695170640890681, 

[CV 1/5; 1/500] START colsample_bylevel=0.9711041112967383, colsample_bynode=0.8260058929866001, colsample_bytree=0.8910774836315021, gamma=0.9097375308994864, learning_rate=0.09241782445712447, max_depth=23, n_estimators=56, reg_alpha=1.468026767557701, reg_lambda=1.8702812504091426, subsample=0.8123740778035529
[CV 1/5; 1/500] END colsample_bylevel=0.9711041112967383, colsample_bynode=0.8260058929866001, colsample_bytree=0.8910774836315021, gamma=0.9097375308994864, learning_rate=0.09241782445712447, max_depth=23, n_estimators=56, reg_alpha=1.468026767557701, reg_lambda=1.8702812504091426, subsample=0.8123740778035529;, score=(train=0.953, test=0.846) total time=   0.5s
[CV 4/5; 3/500] START colsample_bylevel=0.7878013965007412, colsample_bynode=0.9427081886804795, colsample_bytree=0.798046078686356, gamma=1.9858833025592966, learning_rate=0.0597981760366163, max_depth=28, n_estimators=91, reg_alpha=0.4397954985023893, reg_lambda=1.400993978629149, subsample=0.9068558859224319
[CV 4/

[CV 5/5; 26/500] START colsample_bylevel=0.7307334105057043, colsample_bynode=0.6725343493553646, colsample_bytree=0.7046037340151936, gamma=1.8640576081222027, learning_rate=0.14152913976050138, max_depth=28, n_estimators=62, reg_alpha=0.7318255752158997, reg_lambda=1.6158011093191331, subsample=0.8480917963225092
[CV 5/5; 26/500] END colsample_bylevel=0.7307334105057043, colsample_bynode=0.6725343493553646, colsample_bytree=0.7046037340151936, gamma=1.8640576081222027, learning_rate=0.14152913976050138, max_depth=28, n_estimators=62, reg_alpha=0.7318255752158997, reg_lambda=1.6158011093191331, subsample=0.8480917963225092;, score=(train=0.924, test=0.831) total time=   0.2s
[CV 3/5; 28/500] START colsample_bylevel=0.9677331560915355, colsample_bynode=0.9159727449007148, colsample_bytree=0.6844127755851845, gamma=0.0368223366756093, learning_rate=0.09679896765528324, max_depth=46, n_estimators=149, reg_alpha=0.09328050233317753, reg_lambda=0.8684917306160786, subsample=0.7681914204029

[CV 3/5; 1/500] START colsample_bylevel=0.9711041112967383, colsample_bynode=0.8260058929866001, colsample_bytree=0.8910774836315021, gamma=0.9097375308994864, learning_rate=0.09241782445712447, max_depth=23, n_estimators=56, reg_alpha=1.468026767557701, reg_lambda=1.8702812504091426, subsample=0.8123740778035529
[CV 3/5; 1/500] END colsample_bylevel=0.9711041112967383, colsample_bynode=0.8260058929866001, colsample_bytree=0.8910774836315021, gamma=0.9097375308994864, learning_rate=0.09241782445712447, max_depth=23, n_estimators=56, reg_alpha=1.468026767557701, reg_lambda=1.8702812504091426, subsample=0.8123740778035529;, score=(train=0.948, test=0.812) total time=   0.5s
[CV 5/5; 3/500] START colsample_bylevel=0.7878013965007412, colsample_bynode=0.9427081886804795, colsample_bytree=0.798046078686356, gamma=1.9858833025592966, learning_rate=0.0597981760366163, max_depth=28, n_estimators=91, reg_alpha=0.4397954985023893, reg_lambda=1.400993978629149, subsample=0.9068558859224319
[CV 5/

[CV 3/5; 2/500] START colsample_bylevel=0.9787261129575325, colsample_bynode=0.8677673304225337, colsample_bytree=0.8782474373548848, gamma=1.5770092792400032, learning_rate=0.10852981001424009, max_depth=43, n_estimators=91, reg_alpha=0.09270775835686051, reg_lambda=0.8080326197405288, subsample=0.9158961594248594
[CV 3/5; 2/500] END colsample_bylevel=0.9787261129575325, colsample_bynode=0.8677673304225337, colsample_bytree=0.8782474373548848, gamma=1.5770092792400032, learning_rate=0.10852981001424009, max_depth=43, n_estimators=91, reg_alpha=0.09270775835686051, reg_lambda=0.8080326197405288, subsample=0.9158961594248594;, score=(train=0.975, test=0.821) total time=   0.5s
[CV 4/5; 2/500] START colsample_bylevel=0.9787261129575325, colsample_bynode=0.8677673304225337, colsample_bytree=0.8782474373548848, gamma=1.5770092792400032, learning_rate=0.10852981001424009, max_depth=43, n_estimators=91, reg_alpha=0.09270775835686051, reg_lambda=0.8080326197405288, subsample=0.915896159424859

...

[CV 2/5; 417/500] START colsample_bylevel=0.6554410307807731, colsample_bynode=0.9121801080044503, colsample_bytree=0.809876745513442, gamma=1.1393839474260141, learning_rate=0.08309546648289978, max_depth=43, n_estimators=51, reg_alpha=0.29237948011353887, reg_lambda=1.1961395254755456, subsample=0.7323262920896425
[CV 2/5; 417/500] END colsample_bylevel=0.6554410307807731, colsample_bynode=0.9121801080044503, colsample_bytree=0.809876745513442, gamma=1.1393839474260141, learning_rate=0.08309546648289978, max_depth=43, n_estimators=51, reg_alpha=0.29237948011353887, reg_lambda=1.1961395254755456, subsample=0.7323262920896425;, score=(train=0.963, test=0.815) total time=   0.5s
[CV 1/5; 419/500] START colsample_bylevel=0.6520717851448238, colsample_bynode=0.9785322147707682, colsample_bytree=0.7240444484858947, gamma=1.3050513316267482, learning_rate=0.1279265475420549, max_depth=40, n_estimators=118, reg_alpha=0.9043029086241503, reg_lambda=1.9347119489744697, subsample=0.820705506365

[CV 4/5; 435/500] START colsample_bylevel=0.8272302350280604, colsample_bynode=0.9388929095094593, colsample_bytree=0.6321715726868778, gamma=1.7194231910037912, learning_rate=0.05141604198353644, max_depth=43, n_estimators=135, reg_alpha=1.6565262847934095, reg_lambda=1.0708833082881326, subsample=0.9607627194689028
[CV 4/5; 435/500] END colsample_bylevel=0.8272302350280604, colsample_bynode=0.9388929095094593, colsample_bytree=0.6321715726868778, gamma=1.7194231910037912, learning_rate=0.05141604198353644, max_depth=43, n_estimators=135, reg_alpha=1.6565262847934095, reg_lambda=1.0708833082881326, subsample=0.9607627194689028;, score=(train=0.919, test=0.831) total time=   0.5s
[CV 2/5; 437/500] START colsample_bylevel=0.972328564876296, colsample_bynode=0.7981064585828137, colsample_bytree=0.811759179462853, gamma=0.160153710817863, learning_rate=0.12055348249416438, max_depth=29, n_estimators=52, reg_alpha=0.28803384390119513, reg_lambda=1.6108855675375051, subsample=0.664375394647

[CV 4/5; 432/500] START colsample_bylevel=0.9830046000795366, colsample_bynode=0.7462774801128785, colsample_bytree=0.8317886154617946, gamma=0.8600733805348204, learning_rate=0.07751430938127443, max_depth=21, n_estimators=71, reg_alpha=0.6711334790762788, reg_lambda=1.721385754045323, subsample=0.7164538888601761
[CV 4/5; 432/500] END colsample_bylevel=0.9830046000795366, colsample_bynode=0.7462774801128785, colsample_bytree=0.8317886154617946, gamma=0.8600733805348204, learning_rate=0.07751430938127443, max_depth=21, n_estimators=71, reg_alpha=0.6711334790762788, reg_lambda=1.721385754045323, subsample=0.7164538888601761;, score=(train=0.966, test=0.832) total time=   0.7s
[CV 3/5; 435/500] START colsample_bylevel=0.8272302350280604, colsample_bynode=0.9388929095094593, colsample_bytree=0.6321715726868778, gamma=1.7194231910037912, learning_rate=0.05141604198353644, max_depth=43, n_estimators=135, reg_alpha=1.6565262847934095, reg_lambda=1.0708833082881326, subsample=0.9607627194689

[CV 5/5; 434/500] START colsample_bylevel=0.777664913292561, colsample_bynode=0.6687578351493711, colsample_bytree=0.6067330959275045, gamma=0.575445087382126, learning_rate=0.09798728888325355, max_depth=19, n_estimators=68, reg_alpha=0.22726513087024003, reg_lambda=1.1624436401112659, subsample=0.8677755486961419
[CV 5/5; 434/500] END colsample_bylevel=0.777664913292561, colsample_bynode=0.6687578351493711, colsample_bytree=0.6067330959275045, gamma=0.575445087382126, learning_rate=0.09798728888325355, max_depth=19, n_estimators=68, reg_alpha=0.22726513087024003, reg_lambda=1.1624436401112659, subsample=0.8677755486961419;, score=(train=0.993, test=0.832) total time=   0.5s
[CV 1/5; 436/500] START colsample_bylevel=0.997537408809501, colsample_bynode=0.6674418201917904, colsample_bytree=0.9305409675030965, gamma=0.27917775137184786, learning_rate=0.1497784841250334, max_depth=26, n_estimators=112, reg_alpha=1.2568157788339487, reg_lambda=0.9366678362270608, subsample=0.65677232946569

[CV 3/5; 445/500] START colsample_bylevel=0.9067489477420054, colsample_bynode=0.8793762434456138, colsample_bytree=0.702261675741174, gamma=1.7126260783191798, learning_rate=0.12946408726029873, max_depth=41, n_estimators=121, reg_alpha=1.8272349806384305, reg_lambda=1.936696516006309, subsample=0.8601689107097614
[CV 3/5; 445/500] END colsample_bylevel=0.9067489477420054, colsample_bynode=0.8793762434456138, colsample_bytree=0.702261675741174, gamma=1.7126260783191798, learning_rate=0.12946408726029873, max_depth=41, n_estimators=121, reg_alpha=1.8272349806384305, reg_lambda=1.936696516006309, subsample=0.8601689107097614;, score=(train=0.912, test=0.809) total time=   0.3s
[CV 5/5; 446/500] START colsample_bylevel=0.6722523187995383, colsample_bynode=0.9324609550001397, colsample_bytree=0.9768837944644293, gamma=0.06623196956969823, learning_rate=0.1341212538453745, max_depth=15, n_estimators=76, reg_alpha=0.9864467745637417, reg_lambda=0.2890613715917698, subsample=0.80468371338693

In [ ]:
cv_results.head()

In [ ]:
results_path = f"./tuning_results/tuning_xgb/{timestamp}/Assets"
if not os.path.exists(results_path):
    os.makedirs(results_path)

# CV Evaluation

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
cv_results.sort_values(by='rank_test_score', ascending=True).head(5)

In [ ]:
sorted_cv = cv_results.sort_values(by='rank_test_score', ascending=True)

# Train vs Test Comparison

In [ ]:
plt.figure(figsize=(16, 6))    

plt.plot(sorted_cv['rank_test_score'], sorted_cv['mean_train_score'], label="Train Score")
plt.plot(sorted_cv['rank_test_score'], sorted_cv['mean_test_score'], label="Validation Score")

plt.grid()
plt.xlabel('Sorted Validation Rank')
plt.ylabel('Accuracy')
plt.title('Train and Test Accuracy by Final Rank')
plt.legend(loc='best')

filename = "test_train_by_rank.png"
plt.savefig(os.path.join(results_path, filename))

plt.show()

In [ ]:
# boxplot algorithm comparison
fig = plt.figure(figsize=(10, 3))
fig.suptitle('Test Accuracy by Rank')

ax = fig.add_subplot(111)

plt.boxplot(sorted_cv.iloc[:10, :][['split0_test_score', 'split1_test_score', 'split2_test_score',
   'split3_test_score', 'split4_test_score']].T)
ax.set_xticklabels(range(1, 11))
ax.set_xlabel("Rank")
ax.set_ylabel("Accuracy")

filename = "test_accuracy_by_rank.png"
plt.savefig(os.path.join(results_path, filename))

plt.show()

In [ ]:
# boxplot algorithm comparison
fig = plt.figure(figsize=(10, 3))
fig.suptitle('Train Accuracy by Rank')

ax = fig.add_subplot(111)

plt.boxplot(sorted_cv.iloc[:10, :][['split0_train_score', 'split1_train_score', 'split2_train_score',
   'split3_train_score', 'split4_train_score']].T)
ax.set_xticklabels(range(1, 11))
ax.set_xlabel("Rank")
ax.set_ylabel("Accuracy")

filename = "train_accuracy_by_rank.png"
plt.savefig(os.path.join(results_path, filename))

plt.show()

In [ ]:
max_params = cv_results.loc[cv_results['rank_test_score'] == 1]
best_params = max_params.params.values[0]

In [ ]:
print(f"Mean Train Accuracy = {max_params['mean_train_score'].values[0]:.2f}")
print(f"Mean Validation Accuracy = {max_params['mean_test_score'].values[0]:.2f}")

In [ ]:
random_search = pickle.load(open(os.path.join(f'./tuning_results/tuning_xgb/{timestamp}/', file_name), "rb"))
model = random_search.best_estimator_

#model = XGBClassifier(**best_params)
#model.fit(X_train, y_train)

y_train_prediction = model.predict(X_train)
# y_test_prediction = model.predict(X_test)

In [ ]:
print(f"Train set, Accuracy = {accuracy_score(y_train, y_train_prediction):.2f}")
# print(f"Test set, Accuracy = {accuracy_score(y_test, y_test_prediction):.2f}")

In [ ]:
ind = np.argpartition(model.feature_importances_, -20)[-20:]

features = X.columns[ind]
importance = model.feature_importances_[ind]

fig = plt.figure(figsize=(12, 6))
plt.barh(range(len(ind)), importance, align='center')
plt.yticks(range(len(ind)), features)
plt.title('Feature Importance XGB')
plt.grid()

filename = "feature_importance.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()

In the context of XGBoost, these scores are often computed based on the following:

- Weight: The number of times a feature appears in a tree across the ensemble of trees.
- Gain: The average gain of a feature when it is used in trees.
- Cover: The average coverage of a feature when it is used in trees.

In [ ]:
# TEST
for col in ['param_colsample_bylevel', 'param_colsample_bynode',
       'param_colsample_bytree', 'param_gamma', 'param_learning_rate',
       'param_max_depth', 'param_n_estimators', 'param_reg_alpha',
       'param_reg_lambda', 'param_subsample']:
    
    plt.figure(figsize=(16, 6))    

    m, b = np.polyfit(list(sorted_cv['mean_test_score'].values), list(sorted_cv[col].values), 1)
    plt.plot(sorted_cv['mean_test_score'], m * sorted_cv['mean_test_score'] + b, c='r', label="Regression Line")
    plt.scatter(sorted_cv['mean_test_score'], sorted_cv[col], label=f"{col} Values")
    
    plt.grid()
    plt.xlabel('Mean Validation Score')
    plt.ylabel('Parameter Value')
    plt.title(col)
    plt.legend(loc='best')

    
    filename = f"{col}_by_rank.png"
    plt.savefig(os.path.join(results_path, filename))
                  
    plt.show()


# Hyperparameter Evaluation

In [ ]:

def plot_parameters(x_values, title):
    
    fig, ax1 = plt.subplots(figsize=(16, 6))
    ax2 = ax1.twinx()

    ax1.scatter(x_values, cv_results['mean_test_score'], label='mean_test_score', c='b')
    #ax2.scatter(x_values, cv_results['std_test_score'], label='std_test_score', c='r')

    m, b = np.polyfit(list(x_values.values), list(cv_results['mean_test_score'].values), 1)
    ax1.plot(x_values, m * x_values + b, c='b')

    m, b = np.polyfit(list(x_values.values), list(cv_results['std_test_score'].values), 1)
    ax2.plot(x_values, m * x_values + b, c='r', label='std_test_score')
    
    ax1.set_title(title)
    ax1.set_xlabel('Parameter Value')
    ax1.set_ylabel('Mean Test Score')
    ax2.set_ylabel('Standard Deviation of Test Score')
    ax1.grid(True)
    
    
    # Combine the legends from both axes
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2, loc='upper right')

    filename = f"{title}_test_score.png"
    plt.savefig(os.path.join(results_path, filename))
            
    plt.show()



In [ ]:
for param in ['param_colsample_bylevel', 'param_colsample_bynode', 'param_colsample_bytree',
                     'param_gamma', 'param_learning_rate', 'param_max_depth', 'param_n_estimators', 
                     'param_reg_alpha', 'param_reg_lambda', 'param_subsample']:
    x_values = cv_results[param]
    plot_parameters(x_values, param)

# Plotting Evaluation Metrics (Precision, Recall, F1-Score, AUC-ROC):


In [ ]:

# For multiclass classification, you need to binarize the labels
y_true_bin = label_binarize(y_train, classes=np.unique(y_train))
y_score_bin = label_binarize(y_train_prediction, classes=np.unique(y_train_prediction))

auc_roc = roc_auc_score(y_true_bin, y_score_bin, average='macro')

classes = len(y_train.unique())

# Plot Precision-Recall curve for each class
precision = dict()
recall = dict()

plt.figure(figsize=(16, 6))    
for i in range(classes):
    precision[i], recall[i], _ = precision_recall_curve(y_true_bin[:, i], y_score_bin[:, i])
    plt.plot(recall[i], precision[i], label='Covertype {}'.format(i + 1))

plt.grid()
plt.xlabel('Recall')
plt.ylabel('True Positive Rate / Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='best')

filename = "precision_recall.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()


# Plot AUC-ROC curve for each class
fpr = dict()
tpr = dict()

plt.figure(figsize=(16, 6))    
for i in range(classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_score_bin[:, i])
    plt.plot(fpr[i], tpr[i], label='Covertype {}'.format(i + 1))

plt.grid()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate / Precision')
plt.title('ROC Curve')
plt.legend(loc='best')
            
filename = "roc_curve.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()


# Partial Dependence

In [ ]:
# potentially iterate over features (and relation ie 0 to 1)

In [ ]:

features, feature_names = [(0,)], [f"Features #{i}" for i in range(X.shape[1])]
deciles = {0: np.linspace(0, 1, num=5)}

pd_results = partial_dependence(
    model, X, features=1, kind="average", grid_resolution=5)

display = PartialDependenceDisplay(
    [pd_results], features=features, feature_names=feature_names,
    target_idx=0, deciles=deciles
)
display.plot(pdp_lim={1: (-1.38, 0.66)})

plt.grid()
plt.xlabel('Feature Value')
plt.ylabel('Partial Dependence') 
plt.title('Partial Dependence')

filename = "partial_dependence.png"
plt.savefig(os.path.join(results_path, filename))
            
plt.show()


# Confusion Matrix

In [ ]:

class_names = np.unique(y_train)

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
titles_options = [
    ("Confusion matrix without normalization", None),
    ("Normalized confusion matrix", "true"),
]
for title, normalize in titles_options:
    disp = ConfusionMatrixDisplay.from_estimator(
        model,
        X_train,
        y_train,
        display_labels=class_names + 1,
        cmap=plt.cm.Blues,
        normalize=normalize,
    )
    disp.ax_.set_title(title)

    png_name = title.lower().replace(" ", "_")
    filename = f"{png_name}.png"
    plt.savefig(os.path.join(results_path, filename))

plt.show()

# Shap Values

In [ ]:
explainer = shap.TreeExplainer(model)
explanation = explainer.shap_values(X_train, check_additivity=False)


In [ ]:
shap.summary_plot(explanation, X_train, plot_type="bar", show=False)

filename = f"shap_summary.png"
plt.savefig(os.path.join(results_path, filename))
plt.close()  

SHAP values show how each feature affects each final prediction, the significance of each feature compared to others, and the model's reliance on the interaction between features.


In [ ]:
# IF WE SCALE: SCALE ONLY TRAIN DATA SET
# BOOSTING ALG:

# - Feature Selection: AUTOMATIC (NON CRITICAL)
# - Scaling: AUTOMATIC
# - Handling Missing Values: AUTOMATIC
# - Outliers: NEED TO DEAL WITH

# XGB Model Selection

In [ ]:
from numpy import sort
thresholds = sort(model.feature_importances_)

In [ ]:
from sklearn.model_selection import cross_val_score

for thresh in thresholds:
    # select features using threshold
    selection = SelectFromModel(model, threshold=thresh, prefit=True)
    select_X_train = selection.transform(X_train)
    
    # train model
    selection_model = XGBClassifier(**best_params)
    selection_model.fit(select_X_train, y_train)

    scores = cross_val_score(selection_model, select_X_train, y_train, cv=5)
    accuracy = np.mean(scores)

    print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))

# KAGGLE Prediction

In [ ]:
test_processed = pd.read_csv('Data/test_engineered.csv')

In [ ]:
test_processed.head()

In [ ]:
test_processed = test_processed.loc[:, [col for col in test_processed if not col.startswith('Cover_Type_')]]
X_kaggle = test_processed.drop(columns=['Aspect_Sector'])
y_kaggle = model.predict(X_kaggle) + 1

In [ ]:
pd.read_csv("Data/Kaggle/full_submission.csv").head()

In [ ]:
test_processed['Cover_Type'] = y_kaggle

In [ ]:
kaggle_submission = test_processed.loc[:, ['Id', 'Cover_Type']]

In [ ]:
kaggle_submission.Cover_Type.value_counts()

In [ ]:
kaggle_submission.to_csv(f'Data/kaggle_submission_{timestamp}.csv', index=False)

In [ ]:
pd.read_csv(f'Data/kaggle_submission_{timestamp}.csv')

In [ ]:
timestamp